In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
txt = open('/content/data.txt', 'r')
txt = txt.read()
txt = txt.replace('\n\n', '')
print(txt)

The man for wisdom’s various arts renown’d,
Long exercised in woes, O Muse! resound;
Who, when his arms had wrought the destined fall
Of sacred Troy, and razed her heaven-built wall,
Wandering from clime to clime, observant stray’d,
Their manners noted, and their states survey’d,
On stormy seas unnumber’d toils he bore,
Safe with his friends to gain his natal shore:
Vain toils! their impious folly dared to prey
On herds devoted to the god of day;
The god vindictive doom’d them never more
(Ah, men unbless’d!) to touch that natal shore.
Oh, snatch some portion of these acts from fate,
Celestial Muse! and to our world relate.Now at their native realms the Greeks arrived;
All who the wars of ten long years survived;
And ’scaped the perils of the gulfy main.
Ulysses, sole of all the victor train,
An exile from his dear paternal coast,
Deplored his absent queen and empire lost.
Calypso in her caves constrain’d his stay,
With sweet, reluctant, amorous delay;
In vain-for now the circling years

In [ ]:
txt = txt.split("\n")

In [ ]:
tk = Tokenizer()
tk.fit_on_texts(txt)

In [ ]:
total_words = len(tk.word_index.keys()) + 1

In [ ]:
txt = tk.texts_to_sequences(txt)

In [ ]:
max_len = 0
for i in txt:
  if len(i) > max_len:
    max_len = len(i)
max_len

19

In [ ]:
len(txt)

3211

In [ ]:
features = []
labels = []

for text in txt:
  for index in range(2, len(text)):
    segment = text[:index]
    segment = pad_sequences([segment], maxlen=max_len, padding='pre')[0]
    features.append(segment[:-1])
    end = segment[-1]
    labels.append(end)

features = np.array(features)
labels = np.array(labels)
labels = tf.keras.utils.to_categorical(labels, num_classes=total_words)

print(features.shape)
print(labels.shape)

(19471, 18)
(19471, 4731)


In [ ]:
labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(total_words, 1000, input_length=max_len-1),
    #can remove conv, but it really helps
    tf.keras.layers.Conv1D(filters=128, kernel_size=5,
                    strides=1, padding="causal",
                    activation="relu",
                    input_shape=[None, 1]),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(600, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(600, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(600)),
    tf.keras.layers.Dense(total_words, activation="softmax")
])

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
model.fit(features, labels, epochs=15, verbose=1)

Epoch 1/15
609/609 [==============================] - 43s 71ms/step - loss: 1.1180 - accuracy: 0.7626
Epoch 2/15
609/609 [==============================] - 43s 71ms/step - loss: 1.0794 - accuracy: 0.7738
Epoch 3/15
609/609 [==============================] - 44s 72ms/step - loss: 1.0321 - accuracy: 0.7848
Epoch 4/15
609/609 [==============================] - 43s 71ms/step - loss: 0.9826 - accuracy: 0.7960
Epoch 5/15
609/609 [==============================] - 43s 71ms/step - loss: 0.9993 - accuracy: 0.7886
Epoch 6/15
609/609 [==============================] - 43s 71ms/step - loss: 1.0282 - accuracy: 0.7804
Epoch 7/15
609/609 [==============================] - 43s 71ms/step - loss: 1.0465 - accuracy: 0.7715
Epoch 8/15
609/609 [==============================] - 43s 70ms/step - loss: 1.0105 - accuracy: 0.7811
Epoch 9/15
609/609 [==============================] - 42s 70ms/step - loss: 0.9592 - accuracy: 0.7958
Epoch 10/15
609/609 [==============================] - 42s 70ms/step - loss: 0.932

In [ ]:
def generate_texts(base, length):
  sentence = base
  probs = []
  for i in range(length):
    print(i)
    tokenized = tk.texts_to_sequences([sentence])
    window = pad_sequences(tokenized, maxlen=max_len-1)
    prediction = model.predict(window)
    prediction_index = np.argmax(prediction[0], axis=-1)
    probs.append(max(prediction[0]))
    new_word = list(tk.word_index.keys())[np.random.choice(len(prediction[0]), p=prediction[0])]
    
    sentence += " " + new_word

  for i in range(0, len(sentence)-50, 50):
    print(sentence[i:i+50])
  # print(sentence)

generate_texts("The chief, and dash’d him on the craggy shore", 100)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
The chief, and dash’d him on the craggy shore her 
thus after chiefs searching attends no powers shor
e thus coast my destroy public green view and whis
tling and and voice and thee spoke and affords and
 men our father’s host with sire your telemachus f
orm at and ensue a spacious round and whilst train
 and well if part on from jove fate men men it you
r calm fears men when jove is naval thus two two e
merging on and best my future mould and watery eac
h pavement is train men rites with next my honour’
d foes to ensue streaming heaven his kind ithacus 
